In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


In [3]:
torch.manual_seed(4)
np.random.seed(4)

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import random_split, DataLoader



transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Single channel normalization
])



In [6]:
!mkdir -p data

In [7]:

# dataset = torchvision.datasets.Caltech101(root='./data',
#                                         download=True, transform=transform)

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 16

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Files already downloaded and verified
Files already downloaded and verified


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(train_loader)
images, labels = next(dataiter)

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))

In [ ]:
from utils.visualization import plot_random_samples


# Plot random samples from the training dataset
plot_random_samples(trainset, classes, fig_size=(5, 5), n=16)


In [10]:

import torch
from utils.model_utils import init_weights
from models.vgg import VGG
from trainer import Trainer

In [11]:

num_classes = len(classes)
model = VGG(num_classes=num_classes)
model.apply(init_weights)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-3)

epochs = 10


In [12]:
trainer = Trainer(model, 
                       criterion, 
                       optimizer, 
                       train_loader=train_loader, 
                       val_loader=test_loader, 
                       device="mps",
                       save_path="training_state",
                       save_period=-1)


In [ ]:
trainer.train(epochs)